# COURSEWORK

In [7]:
USE Accounting;

Commands completed successfully.

Total execution time: 00:00:00.003

In [2]:
CREATE TABLE Departments (
	DepartmentID Int Primary Key,
	DepartmentName Varchar(30) NOT NULL UNIQUE
);

Commands completed successfully.

Total execution time: 00:00:00.073

In [3]:
CREATE TABLE PersonalData (
	PersonID Int Not NULL PRIMARY KEY,
	Surname VARCHAR(15) NOT NULL,
	Name VARCHAR(15) NOT NULL,
	MiddleName VARCHAR(15) NOT NULL,
	INN VARCHAR(15) Not NULL UNIQUE,
	DepartmentID INT Not NULL,
	CONSTRAINT fk_PD_DepartmentID FOREIGN KEY (DepartmentID) 
		REFERENCES Departments(DepartmentID) ON DELETE CASCADE,
	CONSTRAINT uc_PersonalData UNIQUE(PersonID, Surname, Name, MiddleName)
);

Commands completed successfully.

Total execution time: 00:00:00.122

In [4]:
CREATE TABLE PassportData (
	PassportDataID INT NOT NULL PRIMARY KEY, 
	SPassport   VARCHAR(10)  NOT NULL,
	NPassport   INT  NOT NULL,
	DatePassport date  NOT NULL,
	NameDeptPassport VARCHAR(35)  NOT NULL,
	--PersonID INT NOT NULL,
	CONSTRAINT fk_PD_PasportDataID FOREIGN KEY (PassportDataID) 
		REFERENCES PersonalData(PersonID) ON DELETE CASCADE,
	CONSTRAINT PasportData_Unique UNIQUE (SPassport, NPassport, DatePassport, NameDeptPassport)
);

Commands completed successfully.

Total execution time: 00:00:00.013

In [5]:
CREATE TABLE Positions (
	PositionID Int Primary Key,
	PositionName Varchar(20) NOT NULL UNIQUE
);

Commands completed successfully.

Total execution time: 00:00:00.042

In [8]:
CREATE TABLE WorkCard (
	WorkCardID INT not null PRIMARY KEY,
	--PersonID INT NOT NULL,
	WCNumberOrder INT,
	Organization varchar(20),
	PositionID INT NOT NULL,
	StartDateOfContract date NOT NULL,
	FinishDateOfContract date NOT NULL,
	constraint check_Dates check(FinishDateOfContract > StartDateOfContract OR FinishDateOfContract IS NULL),
	CONSTRAINT fk_JBD_PositionID FOREIGN KEY (PositionID) 
		REFERENCES Positions(PositionID) ON DELETE CASCADE,
	CONSTRAINT fk_JBD_PersonID FOREIGN KEY (WorkCardID)
		REFERENCES PersonalData(PersonID) ON DELETE CASCADE
);

Commands completed successfully.

Total execution time: 00:00:00.219

In [13]:
CREATE TABLE TeacherData (
	TeacherDataID Int Primary Key,
	--PersonID INT NOT NULL,
	PlanTimeOfWork INT check(PlanTimeOfWork between 0 and 10000),
	Experience INT check(Experience between 0 and 10000),
	CONSTRAINT fk_TD_PersonID FOREIGN KEY (TeacherDataID) 
		REFERENCES PersonalData(PersonID) ON DELETE CASCADE,
);

Commands completed successfully.

Total execution time: 00:00:00.013

In [17]:
CREATE TABLE Subjects (
	SubjectID Int Primary Key,
	SubjectName Varchar(20)
);

Commands completed successfully.

Total execution time: 00:00:00.075

In [3]:
CREATE TABLE TeacherSubject (
	SubjectID Int NOT NULL,
	TeacherDataID Int  NOT NULL,
	CONSTRAINT TeachSubPrimary  PRIMARY KEY (SubjectID, TeacherDataID),
	CONSTRAINT fk_TeachSub_TeacherDataID FOREIGN KEY (TeacherDataID) 
		REFERENCES TeacherData(TeacherDataID) ON DELETE CASCADE,
	CONSTRAINT fk_TeachSub_SubjectID FOREIGN KEY (SubjectID) 
		REFERENCES Subjects(SubjectID) ON DELETE CASCADE
);

Commands completed successfully.

Total execution time: 00:00:00.014

In [14]:
/*Список преподавателей, которые работают на определенной кафедре,
с указанием их категории 
(доцент, ассистент, ассистент к.н., профессор, старший преподаватель)
и стажа преподавательской работы?
*/
select Surname, Name, DepartmentName, PositionName, Experience from TeacherData T
JOin PersonalData Per ON Per.PersonID=T.TeacherDataID
	JOIN Departments Dep ON Per.DepartmentID= Dep.DepartmentID 
	JOIN WorkCard Job ON Job.WorkCardID=Per.PersonID
	Join Positions Pos ON Job.PositionID=Pos.PositionID
	where  DepartmentName = 'applied math'
	order by Experience

(0 rows affected)

Total execution time: 00:00:00.048

Surname,Name,DepartmentName,PositionName,Experience


In [15]:
--средняя нагрузка ассистентов указанной кафедры
select Surname, Name, Avg(T.PlanTimeOfWork) avg_time from TeacherData T
JOin PersonalData Per ON Per.PersonID=T.TeacherDataID
	JOIN Departments Dep ON Per.DepartmentID= Dep.DepartmentID 
	JOIN WorkCard Job ON Job.WorkCardID=Per.PersonID
	Join Positions Pos ON Job.PositionID=Pos.PositionID
	where (PositionName = 'assistant' and Dep.DepartmentName = 'applies physics')
	Group By Surname, Name

(0 rows affected)

Total execution time: 00:00:00.062

Surname,Name,avg_time


In [0]:
--дисциплины которые читает каждый из доцентов указанной кафедры
select SubjectName,pd.name,pd.Surname from Subjects s
JOin TeacherAndSubject ts ON ts.SubjectID=s.SubjectID
	JOIN TeacherData td  ON td.TeacherDataID= ts.SubjectID 
	JOIN PersonalData pd ON td.TeacherDataID=pd.PersonID
	Join WorkCard wc ON wc.WorkCardID=pd.PersonID
	join Departments dep on dep.DepartmentID = pd.DepartmentID
	join Positions pos on wc.PositionID = pos.PositionID
	where (PositionName = 'lecturer'  and dep.DepartmentName = 'theoretical physics')

In [0]:
--количество преподавателей из каждой категории
select pos.PositionName , count(*) count_teacher from TeacherData td 
join PersonalData pd on td.TeacherDataID = pd.PersonID
join WorkCard wc ON wc.WorkCardID = pd.PersonID
join Positions pos on wc.PositionID = pos.PositionID
group by pos.PositionName

In [0]:
--истек срок контракта 
select pd.Name,pd.Surname from TeacherData td 
join PersonalData pd on td.TeacherDataID = pd.PersonID
join WorkCard wc ON wc.WorkCardID = pd.PersonID
where wc.FinishDateOfContract<CAST(GETDATE() AS DATE)

In [0]:
--дольше других работает
select pd.Name,pd.Surname,wc.StartDateOfContract from TeacherData td 
join PersonalData pd on td.TeacherDataID = pd.PersonID
join WorkCard wc ON wc.WorkCardID = pd.PersonID
where wc.StartDateOfContract = 
(select min(wc.StartDateOfContract) from WorkCard wc)

In [0]:
--кто из преподавателей может читать указанную дисциплину
select Surname, Name from TeacherData T
JOin PersonalData Per ON Per.PersonID=T.TeacherDataID
	JOIN TeacherAndSubject TS ON T.TeacherDataID = TS.TeacherDataID 
	JOIN Subjects Sub ON Sub.SubjectID= TS.SubjectID
	where (Sub.SubjectName = 'cs')

In [0]:
CREATE PROC WhoCan(
	  @subjectName VARCHAR(35))
AS
IF  NOT EXISTS (SELECT s.SubjectName FROM Subjects s
WHERE s.SubjectName=@subjectName)
SELECT 'Subject does not exist' AS RESULT
ELSE
select Surname, Name from TeacherData T
JOin PersonalData Per ON Per.PersonID=T.TeacherDataID
	JOIN TeacherAndSubject TS ON T.TeacherDataID = TS.TeacherDataID 
	JOIN Subjects Sub ON Sub.SubjectID= TS.SubjectID
	where (Sub.SubjectName = @subjectName)

DROP PROC WhoCan

In [0]:
--видалити співробітника 
CREATE PROC DelWorker(
	  @Name VARCHAR(35),
	  @INN VARCHAR(15))
AS
IF  NOT EXISTS (SELECT INN FROM PersonalData
WHERE INN=@INN)
SELECT 'Person does not exist' AS RESULT
ELSE
delete from PersonalData
where INN = @INN


DROP PROC DelWorker


--exec DelWorker 'Lee','1230123000'

In [0]:
--видалити співробітника 
CREATE PROC AddWorker(
	  @surname VARCHAR(35),
	  @name VARCHAR(35),
	  @middleName VARCHAR(35),
	  @INN VARCHAR(15),
	  @DepId INT,
	  @Spassport VARCHAR(10),
	  @Npassport INT,
	  @datepassport date,
	  @nameDeppassport VARCHAR(35),
	  @WCNumberOrder INT,
	  @org VARCHAR(20),
	  @posID INT,
	  @finishDate date
	  )
AS
IF EXISTS (SELECT INN FROM PersonalData
WHERE INN=@INN)
SELECT 'Person already exists' AS RESULT
ELSE
DECLARE @PID int;
select @PID = (select max(PersonID) from PersonalData) + 1;
INSERT INTO PersonalData VALUES(@PID,@surname,@name,@middleName,@INN,@DepId);
INSERT INTO PassportData VALUES(@PID,@Spassport,@Npassport,@datepassport,@nameDeppassport);
DECLARE @StartDate date;
select @StartDate = CAST(GETDATE() AS DATE);
INSERT INTO WorkCard VALUES(@PID,@WCNumberOrder,@org,@posID,@StartDate,@finishDate);


--exec AddWorker 'john1','john1','john1','1112111111',2,2132,'541443','1972-09-21','dep2',234432,'org1',2,'2022-03-14'	


drop proc AddWorker

In [0]:
--оновити дату контракту 
CREATE PROC NewDate(
	  @newDate DATE,
	  @INN VARCHAR(15))
AS
IF  NOT EXISTS (SELECT INN FROM PersonalData
WHERE INN=@INN)
SELECT 'Person does not exist' AS RESULT
ELSE
update WorkCard 
SET FinishDateOfContract = @newDate
where WorkCardID = (select PersonID from PersonalData
where INN = @INN);

--exec NewDate '2030-05-14','1274589111'
DROP PROC NewDate

In [0]:
--оновити должность 
CREATE PROC ChangePos(
	  @newPosName VARCHAR(20),
	  @INN VARCHAR(15))
AS
IF  NOT EXISTS (SELECT INN FROM PersonalData
WHERE INN=@INN)
SELECT 'Person does not exist' AS RESULT
IF NOT EXISTS (SELECT PositionName FROM Positions
WHERE PositionName=@newPosName)
SELECT 'Position does not exist' AS RESULT
ELSE
DECLARE @posID int;
select @posID = (select PositionID from Positions
where PositionName = @newPosName);
DECLARE @workCardID int;
select @workCardID = (select PersonID from PersonalData
where INN = @INN);
update WorkCard 
SET PositionID = @posID
where WorkCardID = @workCardID;

DROP PROC ChangePos